# <center>Анализ базы данных сервиса для чтения книг по подписке</center>

## Описание

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Предоставлена база данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг.

<b>Цель:</b> анализ базы данных для формулировки ценностное предложение для нового продукта.

## База данных

In [1]:
# импорт библиотек
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# параметры подключения к БД (параметры подключения удалены для сохранения конфиденциальности данных)
db_config = {'user': , # имя пользователя
'pwd': , # пароль
'host': ,
'port': , # порт подключения
'db': } # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# функция для выполнения запроса
def sql_query(query):
    con=engine.connect()
    return pd.io.sql.read_sql(sql=text(query), con = con)

In [4]:
# данные таблицы books
query = '''SELECT * FROM books'''
books = sql_query(query)
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [5]:
query_books = '''SELECT COUNT(book_id) FROM books'''
query_authors = '''SELECT COUNT(DISTINCT author_id) FROM books'''
query_date_min = '''SELECT MIN(publication_date::DATE) FROM books'''
query_date_max = '''SELECT MAX(publication_date::DATE) FROM books'''
query_publisher = '''SELECT COUNT(DISTINCT publisher_id) FROM books'''

print('Количество книг:', sql_query(query_books).loc[0, 'count'])
print('Количество авторов:', sql_query(query_authors).loc[0, 'count'])
print(f"Период публикаций:{sql_query(query_date_min).loc[0, 'min']} - {sql_query(query_date_max).loc[0, 'max']}")
print('Количество издательств:', sql_query(query_publisher).loc[0, 'count'])

Количество книг: 1000
Количество авторов: 636
Период публикаций:1952-12-01 - 2020-03-31
Количество издательств: 340


In [6]:
# данные таблицы authors
query = '''SELECT * FROM authors'''
authors = sql_query(query)
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [7]:
query_authors = '''SELECT COUNT(author_id) FROM authors'''
print('Количество авторов:', sql_query(query_authors).loc[0, 'count'])

Количество авторов: 636


In [8]:
# данные таблицы publishers
query = '''SELECT * FROM publishers'''
publishers = sql_query(query)
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [9]:
query_publisher = '''SELECT COUNT(publisher_id) FROM publishers'''
print('Количество авторов:', sql_query(query_publisher).loc[0, 'count'])

Количество авторов: 340


In [10]:
# данные таблицы ratings
query = '''SELECT * FROM ratings'''
ratings = sql_query(query)
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
query_count_rating = '''SELECT COUNT(*) FROM ratings'''''
query_username = '''SELECT COUNT(DISTINCT username) FROM ratings'''''
query_rating_min = '''SELECT MIN(rating) FROM ratings'''
query_rating_max = '''SELECT MAX(rating) FROM ratings'''

print('Количество выставленных оценок:', sql_query(query_count_rating).loc[0, 'count'])
print('Количество авторов:', sql_query(query_username).loc[0, 'count'])
print(f"Период публикаций:{sql_query(query_rating_min).loc[0, 'min']} - {sql_query(query_rating_max).loc[0, 'max']}")

Количество выставленных оценок: 6456
Количество авторов: 160
Период публикаций:1 - 5


In [12]:
# данные таблицы reviews
query = '''SELECT * FROM reviews'''
reviews = sql_query(query)
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [13]:
query_count_reviews = '''SELECT COUNT(*) FROM reviews'''''
query_username = '''SELECT COUNT(DISTINCT username) FROM reviews'''''
print('Количество написанных обзоров:', sql_query(query_count_reviews).loc[0, 'count'])
print('Количество авторов:', sql_query(query_username).loc[0, 'count'])

Количество написанных обзоров: 2793
Количество авторов: 160


<b> Вывод:</b>
База данных состоит из 5 таблиц:

- books - данные о книгах:
    - book_id — идентификатор книги;
    - author_id — идентификатор автора;
    - title — название книги;
    - num_pages — количество страниц;
    - publication_date — дата публикации книги;
    - publisher_id — идентификатор издателя.
- authors - данные об авторах:
    - author_id — идентификатор автора;
    - author — имя автора.
- publishers - данные об издательствах:
    - publisher_id — идентификатор издательства;
    - publisher — название издательства;
- ratings - данные о пользовательских оценках книг:
    - rating_id — идентификатор оценки;
    - book_id — идентификатор книги;
    - username — имя пользователя, оставившего оценку;
    - rating — оценка книги.
- reviews - данные о пользовательских обзорах на книги:
    - review_id — идентификатор обзора;
    - book_id — идентификатор книги;
    - username — имя пользователя, написавшего обзор;
    - text — текст обзора.
    
Содержит информацию о 1000 книга, опубликованные с 1952-12-01 по 2020-03-31, 636 авторах, 340 издательствах. 160 пользователей выставляли оценки и писали обзоры.

## Анализ базы данных

### Количество книг, вышедших после 1 января 2000 года

In [14]:
query = '''SELECT count(*) FROM books
            WHERE DATE_TRUNC('day', publication_date)>='01.01.2000' '''
print('Количество книг, вышедших после 1 января 2000 года:', sql_query(query).loc[0, 'count'])

Количество книг, вышедших после 1 января 2000 года: 821


В базе данных 80% современных книг, выпущенных с 2000 года.

### Количество обзоров и средняя оценка для каждой книги 

In [53]:
query = '''WITH count_review as
                    (
                    SELECT book_id, count(review_id) as count_review FROM reviews
                    GROUP BY book_id
                    ),
                avg_rating as
                    (
                    SELECT book_id, AVG(rating) as avg_rating FROM ratings
                    GROUP BY book_id
                    ),
                    
                final_table as 
                (
                SELECT b.book_id, b.title, avg_rating, COALESCE(count_review, 0) as count_review FROM books as b
                LEFT JOIN avg_rating as ra on b.book_id = ra.book_id
                LEFT JOIN count_review as re on b.book_id = re.book_id
                )
                
                SELECT * FROM final_table
                ORDER BY count_review DESC, avg_rating DESC
            '''

temp = sql_query(query)
temp.head(10)

,book_id,title,avg_rating,count_review
0,948,Twilight (Twilight #1),3.662500,7
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,4.414634,6
2,299,Harry Potter and the Chamber of Secrets (Harry...,4.287500,6
3,656,The Book Thief,4.264151,6
4,734,The Glass Castle,4.206897,6
5,750,The Hobbit or There and Back Again,4.125000,6
6,497,Outlander (Outlander #1),4.125000,6
7,695,The Curious Incident of the Dog in the Night-Time,4.081081,6
8,779,The Lightning Thief (Percy Jackson and the Oly...,4.080645,6
9,963,Water for Elephants,3.977273,6


В ТОП по количеству обзоров попали такие книги, как серии о Гарри Поттере, Хоббит, Сумерки, Перси Джексон и похититель молний и другие. Все они являются бестселлерам или высоко оценнными критиками и были экранизированы. Поэтому остаются обсуждаемыми, а также вызывают разногласия, судя по средним оценкам    

### Издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [18]:
query = '''SELECT p.publisher, COUNT(b.book_id) as count  FROM books as b
           JOIN publishers as p on b.publisher_id = p.publisher_id
           WHERE b.num_pages>50
           GROUP BY p.publisher
           ORDER BY count DESC
           LIMIT 1
           '''
temp = sql_query(query)

print('Издательство, которое выпустило наибольшее число книг толще 50 страниц:', temp.loc[0, 'publisher'])
print('Количество выпущенных книг:', temp.loc[0, 'count'])

Издательство, которое выпустило наибольшее число книг толще 50 страниц: Penguin Books
Количество выпущенных книг: 42


Penguin Books - издательство, которое печатало качественную литературу в мягком переплёте, что позволяло удешевить книги, а значит больше людей имели к ним доступ. Качественная литература востребована всегда. Наличие в лидерах данного издательства говорит об высоком уровне наполнения базы данных.

### Автор с самой высокой средней оценкой книг с учетом книг с 50 и более оценками

In [19]:
query = '''WITH b_r as 
                    (
                    SELECT b.book_id, count(r.rating)  FROM books as b
                    JOIN ratings as r on b.book_id = r.book_id
                    GROUP BY b.book_id
                    HAVING count(r.rating)>=50
                    )
        
           SELECT a.author, AVG(r.rating) as rating FROM books as b
           JOIN ratings as r on b.book_id = r.book_id
           JOIN authors as a on b.author_id = a.author_id
           WHERE b.book_id IN (SELECT book_id FROM b_r)
           GROUP BY a.author
           ORDER BY rating DESC
           LIMIT 1
           '''
temp = sql_query(query)

print('Автор с самой высокой средней оценкой книг:', temp.loc[0, 'author'])
print('Рейтинг:', round(temp.loc[0, 'rating'],2))

Автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré
Рейтинг: 4.29


J.K. Rowling - автор серии книг о Гарри Поттере. Каждая книга серии стала бестселлером: мир волшебства привлек огромное количество поклонников. Эти книги остаются востребованы и у подрастающего поколения.

### Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [20]:
query = '''WITH ra as 
                    (
                    SELECT username, count(rating)  FROM ratings
                    GROUP BY username
                    HAVING count(rating)>48
                    )
            
           SELECT AVG(count) FROM
           (SELECT count(review_id) as count FROM reviews
           WHERE username IN (SELECT username FROM ra)
           GROUP BY username) as re
           '''

print('Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок:', sql_query(query).loc[0, 'avg'])

Cреднее количество обзоров от пользователей, которые поставили больше 48 оценок: 24.0


В базе данных есть информация о пользователях, которые активно читают, ставят оценки и пишут обзоры. Это говорит о востребованности книг, которые есть в базе. Кроме того выставленные оценки и обзоры могут помочь в выборе другим читателям.

## Вывод

База данных состоит из 5 таблиц:

- books - данные о книгах;
- authors - данные об авторах;
- publishers - данные об издательствах;
- ratings - данные о пользовательских оценках книг;
- reviews - данные о пользовательских обзорах на книги.
    
Содержит информацию о 1000 книга, опубликованные с 1952-12-01 по 2020-03-31, 636 авторах, 340 издательствах. 160 пользователей выставляли оценки и писали обзоры.

Были исследованы следующие характеристики:
- количество книг вышло после 1 января 2000 года - 821;
- количество обзоров и средняя оценка для каждой книги;
- издательство, которое выпустило наибольшее число книг толще 50 страниц - Penguin Books;
- автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré;
- среднее количество обзоров от пользователей, которые поставили больше 48 оценок - 24.

В качестве пунктов ценостного предложения можно выделить:

- современнность книг, которые есть в базе данных;
- наличие бестселлеров и качественной литературы;
- большое количество оценок и обзоров для помощи в выборе.
  